# notebook pour l'implémentation de nouvelles fonction pour le module python
notebook pour tester l'implémentation des nouvelles fonctions du module benford.py

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, math
sys.path.insert(0, "..")
import benford as ben


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
data = pd.read_csv("tests/Data_exp/1512006-Calbicans-QUANTI_raw_abundance.csv", sep=",", header=None)
print(data.shape)
data.head()

(2356, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
0,9.325899e+04,2.765206e+04,6.973654e+04,6.309967e+04,2.252522e+04,5.307728e+04,9.088306e+04,2.035561e+04,0.000000e+00,0.000000e+00,1.021926e+05,1.794098e+05
1,5.042612e+07,1.592204e+07,4.388571e+07,1.824970e+07,1.596605e+07,1.566419e+07,2.563818e+07,2.666071e+07,1.431299e+07,2.617207e+07,2.847710e+07,4.051467e+07
2,0.000000e+00,0.000000e+00,6.015709e+03,2.413695e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.132027e+03,4.891291e+04,2.974154e+04,4.261914e+04
3,1.560630e+07,7.262593e+06,4.718013e+06,2.570901e+06,1.629196e+06,2.344451e+06,2.178284e+07,2.715498e+07,1.042588e+06,2.429717e+06,1.749820e+06,2.306792e+06
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.729096e+03,3.448312e+04


In [46]:
# données dans un array d'une dimmenssion
exp_numbers = data.values.reshape(data.shape[0]*data.shape[1])
print(exp_numbers.shape)
exp_numbers = np.array(exp_numbers, dtype=int)

(28272,)


In [47]:
# Remove 0
exp_numbers = exp_numbers[ exp_numbers > 0 ]
exp_numbers.shape

(26919,)

In [48]:
first_digit = ben.count_first_digit(exp_numbers, nb_digit=1)
first_digit

array([7932, 4679, 3266, 2636, 2172, 1881, 1635, 1387, 1282])

In [49]:
first_digit_norm = ben.normalize_first_digit(first_digit)
first_digit_norm

array([0.29519911, 0.17413472, 0.1215482 , 0.09810197, 0.08083364,
       0.07000372, 0.06084853, 0.05161891, 0.0477112 ])

## Fonction POM (Physical Order of Magnitude)

In [51]:
ben.calculate_pom(exp_numbers)

885178948.0

## fonction OOM (Order of Magnitude)

In [54]:
ben.calculate_oom(exp_numbers)

8.94703107665586

## Fonction SSD (Sum of Squares Deviation)

In [55]:
ben.calculate_ssd(first_digit_norm, ben.get_theoretical_freq_benford())

0.7501365778165984

## Fonction RMSSD (Root Mean Sum of Square Deviation)

In [56]:
ben.calculate_rmssd(first_digit_norm, ben.get_theoretical_freq_benford())

0.2887014178113587

## Fonction chisquare test (sans bootstrap)

In [57]:
ben.chi2_test(exp_numbers, ben.get_theoretical_freq_benford())

statistics : 17.35670420146075 ; p-value : 0.026601803104451673


(17.35670420146075, 0.026601803104451673)

#### avec bootstrap

In [69]:
ben.calculate_bootstrap_chi2(exp_numbers, ben.get_theoretical_freq_benford(), nb_digit=1,
                            type_test=0)

statistics : 4.808124805526675 ; p-value : 0.7778733812145178 ; number of significant tests : 26


(4.808124805526675, 0.7778733812145178, 26)

## Fonction G-test (sans bootstrap)

In [70]:
ben.g_test(exp_numbers, ben.get_theoretical_freq_benford())

[7932 4679 3266 2636 2172 1881 1635 1387 1282]
[8103.42645328 4740.20060252 3363.22585076 2608.72064016 2131.47996236
 1802.14063007 1561.08522069 1376.97475176 1231.7458884 ]
statistics : -80.72309844128006 ; p-value : 1.0


(-80.72309844128006, 1.0)

# Test sur la taille des protéines de swissProt

In [74]:
data2 = pd.read_csv("tests/Prot_tot_uniprot/uniprot-reviewed yes.tab", sep="\t",
                   header=0)
print(data.shape)
data.head()

(564638, 1)


,Entry Length
Q8K9I1,960
Q664P8,255
Q66K14,1250
Q8E4B4,239
B3CQ06,426


In [75]:
prot_len = data2["Entry Length"]
prot_len

Q8K9I1     960
Q664P8     255
Q66K14    1250
Q8E4B4     239
B3CQ06     426
          ... 
C3L855     410
P41413    1809
B7HED5     394
B1IUE0     408
Q1KXU1      31
Name: Entry Length, Length: 564638, dtype: int64

In [92]:
# fonction pom :
ben.calculate_pom(prot_len)
# fonction oom :
ben.calculate_oom(prot_len)

POM : 17606.5
POM : 17606.5
OOM : 4.245673031051872


4.245673031051872

In [93]:
d_obs = ben.count_first_digit(prot_len)
# fonction ssd :
ben.calculate_ssd(ben.normalize_first_digit(d_obs), ben.get_theoretical_freq_benford())
# fonction rmssd :
ben.calculate_rmssd(ben.normalize_first_digit(d_obs), ben.get_theoretical_freq_benford())

SDD : 89.04626775928213
SDD : 89.04626775928213
RMSSD : 3.1454776665923645


3.1454776665923645